In [1]:
import numpy as np
from scipy.stats import binom
from ipywidgets import widgets,Layout
import functools

In [2]:
nb_p=6
p_min=0.41
p_max=0.48
v_p=p_min+(p_max-p_min)*((np.arange(nb_p,dtype=np.float64)/(nb_p-1)))[::-1]


def paramt1(nb_of_required_bits,A,v_p):   
    nb_p=v_p.shape[0]
    half_nb_of_required_bits=nb_of_required_bits/2
    v_x2=np.array(half_nb_of_required_bits-A+np.arange(2*A+1),dtype=np.uint64)
    p=0.5
    alpha=1-(binom.pmf(v_x2,nb_of_required_bits,p).sum())
    if alpha<0:
        alpha=0
    v_beta=np.zeros(nb_p,dtype=np.float64)
    for i_p in range(nb_p):
        p=v_p[i_p]
        beta=(binom.pmf(v_x2,nb_of_required_bits,p).sum())
        if beta >0:
            v_beta[i_p]=beta
    return alpha,v_beta

In [6]:
cell_height=60
cell_width=105


w_nb_of_required_bits=widgets.IntSlider(description="Number of inputs bits",style = {'description_width': 'initial'},value=20000,min=4096,max=40000,layout= Layout(width='%dpx'%(cell_width*(nb_p+2))))
w_A=widgets.IntSlider(description="Relative threshold",style = {'description_width': 'initial'},value=346,min=1,max=1000,layout= Layout(width='%dpx'%(cell_width*(nb_p+2))))

alpha,v_beta=paramt1(w_nb_of_required_bits.value,w_A.value,v_p)

v_items=[]

box_layout = Layout(height='%dpx'%cell_height,width='%dpx'%cell_width,border='1px solid gray',margin='0px 0px 0px 0px')
box_layout2 = Layout(height='%dpx'%cell_height,width='%dpx'%(cell_width*nb_p),border='1px solid gray',margin='0px 0px 0px 0px')


v_items_h=[]
v_items_h.append(widgets.HTMLMath('',layout=box_layout))
v_items_h.append(widgets.HTMLMath('<p>Null hypothesis is <b>in fact true</b></p>',layout=box_layout))
v_items_h.append(widgets.HTMLMath('<p>Null hypothesis is <b>in fact false</b></p>',layout=box_layout2))
v_items.append(v_items_h)

v_items_h=[]
v_items_h.append(widgets.HTMLMath('Entropy H<sub>1</sub>',layout=box_layout))
v_items_h.append(widgets.HTMLMath('<p>H<sub>1</sub>=1.00</p>',layout=box_layout))
for i in range(nb_p):
    if ((v_p[i])*(1-v_p[i]))!=0:
        h=-1*(v_p[i])*np.log2(v_p[i])-(1-v_p[i])*np.log2(1-v_p[i])
    else:
        h=0
    v_items_h.append(widgets.HTMLMath('<p> H<sub>1</sub>=%.3f</p>'%h,layout=box_layout))
v_items.append(v_items_h)


v_items_h=[]
v_items_h.append(widgets.HTMLMath('<p>Test <b>rejects</b> Null hypothesis </p>',layout=box_layout))
v_items_h.append(widgets.HTMLMath('<p style="color:#FF8000";><b>&#945=%.3E</b></p>'%alpha,layout=box_layout))
for i in range(nb_p):
    v_items_h.append(widgets.HTMLMath('<p style="color:#00FF00";><b>1-&#946=%.3E</b></p>'%(1-v_beta[i]),layout=box_layout))
v_items.append(v_items_h)

v_items_h=[]
v_items_h.append(widgets.HTMLMath('<p>Test <b>does not reject</b> Null hypothesis</p>',layout=box_layout))
v_items_h.append(widgets.HTMLMath('<p style="color:#00FF00";><b>1-&#945=%.3E</b></p>'%(1-alpha),layout=box_layout))
for i in range(nb_p):
    v_items_h.append(widgets.HTMLMath('<p style="color:#FF0000";><b>&#946=%.3E</b></p>'%(v_beta[i]),layout=box_layout))
v_items.append(v_items_h)

nb_lines=len(v_items)
v_hb=[]
for i in range(nb_lines):
    v_tmp=v_items[i]
    nb_row=len(v_tmp)
    v_box=[]
    for j in range(nb_row):
        v_box.append(v_tmp[j])
    v_hb.append(widgets.HBox(v_box))
v_box_table=widgets.VBox(v_hb)



def change_on_param(obj):
    if obj['new']:
        alpha,v_beta=paramt1(w_nb_of_required_bits.value,w_A.value,v_p)
        (v_box_table.children[2]).children[1].value='<p style="color:#FF8000";><b>&#945=%.3E</b></p>'%alpha
        (v_box_table.children[3]).children[1].value='<p style="color:#00FF00";><b>1-&#945=%.3E</b></p>'%(1-alpha)       
        for i in range(nb_p):
            (v_box_table.children[2]).children[i+2].value='<p style="color:#00FF00";><b>1-&#946=%.3E</b></p>'%(1-v_beta[i])
            (v_box_table.children[3]).children[i+2].value='<p style="color:#FF0000";><b>&#946=%.3E</b></p>'%(v_beta[i])

    
        
        
w_nb_of_required_bits.observe(functools.partial(change_on_param),'value')
w_A.observe(functools.partial(change_on_param),'value')
        
v_box_top=widgets.VBox([w_nb_of_required_bits,w_A,v_box_table])
display(v_box_top)